<a href="https://colab.research.google.com/github/Victor0vich/Denis/blob/main/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_13_%D0%B0%D0%B2%D0%B3%D1%83%D1%81%D1%82%D0%B0_%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE_HR_%D0%B4%D0%BB%D1%8F_%D1%82%D1%80%D1%83%D0%B4%D0%BE%D1%83%D1%81%D1%82%D1%80%D0%BE%D0%B9%D1%81%D1%82%D0%B2%D0%B0_%D0%B2%D1%8B%D0%BF%D1%83%D1%81%D0%BA%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **13-08 Нейро-HR (телеграм-бот)**

**Состав блоков**

* `1 Блок стандартных вопросов` - бот выясняет курс, который кандидат прошел, и задает базовые вопросы для определения критериев по будущему месту работы.

* `2 Блок дополнительных вопросов` - бот проверяет наличие ответов на все вопросы и задает три уточняющих вопроса.

* `3 Блок стажировок` - бот предлагает написать список стажировок, в которых кандидат учавствовал, по ключевым словам из списка стажировок в суммаризированный диалог вставляются оригинальные названия стажировок с их лэндами.

* `4 Блок запроса старого резюме:` бот просит кандидата скинуть имеющееся у него резюме или данные по опыту работы.

* `5 Блок создания шаблонного резюме:` бот на основании представленного резюме и введенной информации собирает шаблонное резюме для кандидата.

* `6 Блок подбора вакансий:` бот на переводит резюме в профайл, и вакансии в профайлы, и на основании векторной базы находит 20 релевантных вакансий, а потом ЧатГПТ выберает 5 лучших и выводит кандидату для выбора.

* `7 Блок корректировки резюме:` бот на основании выбранной вакансии корректирует текущее резюме под эту вакансию.

## **Расшифка рабочих блоков телеграм-бота.**

In [ ]:
#@title Установка, импорт библиотек и активация ключа API
!pip install openai==0.28.1
!pip install requests beautifulsoup4
!pip install faiss-cpu langchain==0.0.271 tiktoken

import os
from getpass import getpass  # Скрытый ввод паролей
import openai  # Импортируем openai
import textwrap
import pandas as pd
from IPython.display import clear_output
from langchain.embeddings.openai import OpenAIEmbeddings  # Импортируем OpenAIEmbeddings

openai_key = getpass("Введите токен OpenAI API Key:")

# Установка токена OpenAI
openai.api_key = openai_key

# Инициализация модели эмбеддингов с использованием уже установленного ключа API
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)  # используем введенный ключ напрямую

# Блок загрузки данных из таблицы
import gspread
from google.colab import auth
from google.auth import default
import re
# Авторизация в Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Блок парсинга
import requests
from bs4 import BeautifulSoup
import json
import time

# Блок загрузки вакансий
import gdown
import zipfile

# Векторная база ФАИС
import faiss
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import shutil

clear_output()

### **1. Блок выбора курса и стандартных вопросов**

In [ ]:
#@title Сервисные функции первого блока

# Функция для форматирования текста
def format_text(text, width=120):
    return '\n'.join(textwrap.fill(line, width) for line in str(text).split('\n'))

# Функция преобразования текста в список строк
def text_to_list_lines(text):
    lines_list = text.split("\n") # список по разделителю переноса строки
    return [line for line in lines_list if line.strip() != ''] # из списка убираем пустые строки

# Функция диалога для выбора пройденного курса
def course_dialog(questions: str) -> str:
    print(format_text(questions), '')
    answer = input("Ваш выбор (1 или 2): ")

    if answer == '1':
        course_name = "Data Science, нейронные сети, машинное обучение и искусственный интеллект"
        course_skills = data_science_stack
    elif answer == '2':
        course_name = "Chat GPT Professional"
        course_skills = gpt_stack
    else:
        print("Неверный ввод, пожалуйста, выберите 1 или 2.")
        return course_dialog(questions)  # Повторный вызов функции для корректного ввода

    # Формируем итоговый диалог без лишних переносов строк
    dialog_str = f"Завершенный курс: {course_name}\nНавыки (стэк): {course_skills}\n"
    return dialog_str

# Функция диалога (вопрос-ответ) и сохранение
def dialog(questions: str) -> str:
    questions_list = text_to_list_lines(questions)  # вопросы из текста в список построчно
    dialog_str = ''
    for i, question in enumerate(questions_list, 1):
        formatted_question = format_text(f"\nБот: {question}")
        print(formatted_question, '')
        answer = input("Ответ: ")
        formatted_answer = format_text(f"Пользователь: {answer}\n")
        # накапливаем историю диалога
        dialog_str += f"{formatted_question}\n{formatted_answer}"
    return dialog_str

In [ ]:
# Стэки курсов
data_science_stack = """
Компьютерное зрение: Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, ImageEnhance, Tesseract, PyTesseract, Stable Diffusion, easyocr, pdf2image, TerraYoloV5, SAM, LangSAM, GroundingDINO, Roboflow, CVAT, Ultralytics
Работа с текстами: NLTK, SpaCy, Transformers (Hugging Face), Langchain, Tiktoken, PyDub, SpeechRecognition, Whisper, Bert, GPT, T5
Машинное обучение и глубокое обучение: TensorFlow, Keras, PyTorch (Torch), Scikit-learn, AutoKeras, H2O.ai, TPOT, Transformers (Hugging Face), Horovod, Stable Diffusion, Diffusers
Анализ данных и визуализация: Pandas, Numpy, Matplotlib, Seaborn, Prophet, AutoTS, MoviePy, VidGear, SciPy
Разработка веб-приложений: Flask, FastAPI, Uvicorn, Gunicorn, Django
Системное администрирование и DevOps: Docker, Docker Compose, Nginx, OS, Shutil, Glob, VSCode, dotenv, python-telegram-bot, pygit2
Обработка звука и речи: SpeechRecognition, PyDub, gTTS, TTS, LibROSA, Jiwer, PyAudio, Whisper, Google Text-to-Speech
Работа с данными и API: Requests, FAISS, OpenAI, Gspread, Google API Client, Telegram API, OpenAI API
Мультимедийная обработка: MoviePy, PIL, pdf2image, VidGear
Прочие навыки и инструменты: Google Colaboratory, Getpass, ImageEnhance, PIL, VSCode, Telegram.ext, dotenv
"""

gpt_stack = """
Работа с API и интеграция: OpenAI API, Telegram API, Google API Client, Faiss, MediaIoBaseDownload
Работа с текстами и обработка данных: ChatGPT, LangChain, Faiss, Telegram API, Whisper, HuggingFace, NLLB, MarianMT, Saiga, Embeddings, Pymorphy2, Tiktoken, Kaleido, LLaMA, Chroma
Машинное обучение и глубокое обучение: TensorFlow, Keras, PyTorch, GPT-4, State of the Art (SoTA) локальные модели, LLM, FFNN, CNN, RNN, GAN, Cohere
Разработка веб-приложений: Flask, Django, FastAPI, Uvicorn, Docker, Android SDK, Java, Kotlin
Обработка звука и речи: SpeechRecognition, PyDub, gTTS, TTS, Whisper
Общие инструменты и библиотеки: Google Colaboratory, Python, Pandas, Numpy, Matplotlib, VSCode, Gspread, Pydantic, TQDM, NestAsyncio, xmltodict, Docx, BeautifulSoup, Unstructured, pdf2image, pdfminer.six, OpenAIEmbeddings, FAISS, RecursiveCharacterTextSplitter, Html2TextTransformer, ChatOpenAI, MultiRetrievalQAChain, UnstructuredPDFLoader, Google API Client, MediaIoBaseDownload, Ast, PdfMiner, VSCode
Прочие навыки и инструменты: VSCode, Ast, PdfMiner, LLaMA, Chroma
"""

# Вопросы для выбора курса
course_question = """
Здравствуйте!
Пожалуйста, выберите курс, который Вы закончили, выбрав цифру 1 или 2.
1. Data Science, нейронные сети, машинное обучение и искусственный интеллект
2. Chat GPT Professional
"""

# Вызов функции диалога для курса
course_summarized_dialog = course_dialog(course_question)

# Вопросы диалога
initial_questions = """
Когда проходили обучение (укажите в формате: 01.2024-06.2024)?
В каком Вы городе находитесь?
Какой формат работы у Вас в приоритете (Офис, Удаленная работа, Гибрид)?
На какой уровень зарплаты расчитываете (руб/мес)?
Вы хотите в крупной компании работать или не принципиально?
Есть ли у Вас пожелание по проектам (CV, NLP, Trading и т.д.)?
"""

# Вызов функции диалога для остальных вопросов
summarized_dialog = dialog(initial_questions)

# Добавляем в начало суммаризированного диалога позицию
summarized_dialog = f"Позиция: AI/ML Разработчик\n\n{course_summarized_dialog}{summarized_dialog}"


Здравствуйте!
Пожалуйста, выберите курс, который Вы закончили, выбрав цифру 1 или 2.
1. Data Science, нейронные сети, машинное обучение и искусственный интеллект
2. Chat GPT Professional
 
Ваш выбор (1 или 2): 1

Бот: Когда проходили обучение (укажите в формате: 01.2024-06.2024)? 
Ответ: 09.2023-05.2024

Бот: В каком Вы городе находитесь? 
Ответ: Челябинск

Бот: Какой формат работы у Вас в приоритете (Офис, Удаленная работа, Гибрид)? 
Ответ: Офис

Бот: На какой уровень зарплаты расчитываете (руб/мес)? 
Ответ: 100 000

Бот: Вы хотите в крупной компании работать или не принципиально? 
Ответ: Не принципиально

Бот: Есть ли у Вас пожелание по проектам (CV, NLP, Trading и т.д.)? 
Ответ: CV


In [ ]:
# Вывод суммарного диалога
print(summarized_dialog)

Позиция: AI/ML Разработчик

Завершенный курс: Data Science, нейронные сети, машинное обучение и искусственный интеллект
Навыки (стэк): 
Компьютерное зрение: Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, ImageEnhance, Tesseract, PyTesseract, Stable Diffusion, easyocr, pdf2image, TerraYoloV5, SAM, LangSAM, GroundingDINO, Roboflow, CVAT, Ultralytics
Работа с текстами: NLTK, SpaCy, Transformers (Hugging Face), Langchain, Tiktoken, PyDub, SpeechRecognition, Whisper, Bert, GPT, T5
Машинное обучение и глубокое обучение: TensorFlow, Keras, PyTorch (Torch), Scikit-learn, AutoKeras, H2O.ai, TPOT, Transformers (Hugging Face), Horovod, Stable Diffusion, Diffusers
Анализ данных и визуализация: Pandas, Numpy, Matplotlib, Seaborn, Prophet, AutoTS, MoviePy, VidGear, SciPy
Разработка веб-приложений: Flask, FastAPI, Uvicorn, Gunicorn, Django
Системное администрирование и DevOps: Docker, Docker Compose, Nginx, OS, Shutil, Glob, VSCode, dotenv, python-telegram-bot, pygit2
Обработка звука 

### **2. Блок дополнительных вопросов**

In [ ]:
#@title Сервисные функции второго блока

# Функция генерации дополнительного вопроса
def generate_additional_question(summarized_dialog):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query.format(summarized_dialog=summarized_dialog)}
        ],
        temperature=0.5,
        max_tokens=100
    )
    return response['choices'][0]['message']['content']

**ПРОМТЫ ДОПОЛНИТЕЛЬНЫХ ВОПРОСОВ**

In [ ]:
# Пример использования функции
prompt = """Ты опытный специалист по подбору персонала. Ты должен
проанализировать суммарный диалог с кандидатом и проверить, что на все вопросы получены ответы.
Если на какой-то вопрос ответа нет или информация неполная, задай один простой уточняющий вопрос кандидату в отношении предпочтений в будущей работе.
Все утоняющие вопросы должны быть уникальными, не повторяйся в вопросах.
"""
query = f"""
Проверьте следующие ответы кандидата: {summarized_dialog}.
Убедитесь, что на все вопросы получены ответы. Если чего-то не хватает или нужно уточнить, задав один простой вопрос.
Напиши только вопрос, без выводов. Все уточняющие вопросы должны быть уникальными, нельзя повторять вопросы.
"""

**ПЕРВЫЙ ВОПРОС**

In [ ]:
# Вызов функции генерации дополнительного вопроса
additional_question = generate_additional_question(summarized_dialog)
print(additional_question)

# Ввод ответа на уточняющий вопрос и обновление summarized_dialog
additional_answer = input("Ваш ответ на уточняющий вопрос: ")
summarized_dialog += f"\nУточняющий вопрос: {additional_question}\nОтвет: {additional_answer}\n"

Какой у Вас опыт работы с проектами в области компьютерного зрения?
Ваш ответ на уточняющий вопрос: Работал над одиним проектом распознавания лиц


**ВТОРОЙ ВОПРОС**

In [ ]:
# Вызов функции генерации дополнительного вопроса
additional_question = generate_additional_question(summarized_dialog)
print(additional_question)

# Ввод ответа на уточняющий вопрос и обновление summarized_dialog
additional_answer = input("Ваш ответ на уточняющий вопрос: ")
summarized_dialog += f"\nУточняющий вопрос: {additional_question}\nОтвет: {additional_answer}\n"

Какой график работы вам предпочтителен (полный день, частичная занятость, проектная работа)?
Ваш ответ на уточняющий вопрос: полный день


**ТРЕТИЙ ВОПРОС**

In [ ]:
# Вызов функции генерации дополнительного вопроса
additional_question = generate_additional_question(summarized_dialog)
print(additional_question)

# Ввод ответа на уточняющий вопрос и обновление summarized_dialog
additional_answer = input("Ваш ответ на уточняющий вопрос: ")
summarized_dialog += f"\nУточняющий вопрос: {additional_question}\nОтвет: {additional_answer}\n"

Какой тип команды или рабочего окружения вам предпочтительнее (стартап, крупная команда, маленькая команда)?
Ваш ответ на уточняющий вопрос: Маленькая команда


**ВЫВОДИМ ТЕКУЩИЙ СУММАРИЗИРОВАННЫЙ ДИАЛОГ**

In [ ]:
print(summarized_dialog)

Позиция: AI/ML Разработчик

Завершенный курс: Data Science, нейронные сети, машинное обучение и искусственный интеллект
Навыки (стэк): 
Компьютерное зрение: Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, ImageEnhance, Tesseract, PyTesseract, Stable Diffusion, easyocr, pdf2image, TerraYoloV5, SAM, LangSAM, GroundingDINO, Roboflow, CVAT, Ultralytics
Работа с текстами: NLTK, SpaCy, Transformers (Hugging Face), Langchain, Tiktoken, PyDub, SpeechRecognition, Whisper, Bert, GPT, T5
Машинное обучение и глубокое обучение: TensorFlow, Keras, PyTorch (Torch), Scikit-learn, AutoKeras, H2O.ai, TPOT, Transformers (Hugging Face), Horovod, Stable Diffusion, Diffusers
Анализ данных и визуализация: Pandas, Numpy, Matplotlib, Seaborn, Prophet, AutoTS, MoviePy, VidGear, SciPy
Разработка веб-приложений: Flask, FastAPI, Uvicorn, Gunicorn, Django
Системное администрирование и DevOps: Docker, Docker Compose, Nginx, OS, Shutil, Glob, VSCode, dotenv, python-telegram-bot, pygit2
Обработка звука 

### **3. Блок по стажировкам**

In [ ]:
#@title Сервисные функции треьего блока

# Функция для загрузки данных из Google Sheets
def load_data():
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ypT_2BhMaVDR8rPJDU7gHdrf2GSsrbE6lfvu7D5NOgk/edit?usp=sharing')
    worksheet = sh.worksheet('Лист1')
    rows = worksheet.get_all_values()
    dataframe = pd.DataFrame(rows[2:], columns=["URL", "Title"])
    return dataframe

# Функция для получения пользовательского ввода
def get_user_input():
    user_input = input("Введите названия стажировок через запятую (или оставьте пустым, если стажировки не проходили): ")
    if user_input.strip():  # Проверка на пустую строку
        return [title.strip() for title in user_input.split(',')]
    else:
        return []

def generate_relevant_internships(user_titles, internships_list):
    # Формирование запроса
    query = query_template.format(user_titles=', '.join(user_titles), internships_list=internships_list)

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        temperature=0,
        max_tokens=500
    )

    internships = response['choices'][0]['message']['content'].split('\n')

    # Дополнительно парсим информацию о стажировках
    detailed_internships = []
    for internship in internships:
        if internship.strip():
            title, url = internship.split(': ')
            details = extract_internship_details(url)
            detailed_internships.append(f"{title}\n{details}\nСсылка: {url}\n")

    return '\n'.join(detailed_internships)

def extract_internship_details(url):
    try:
        # Загружаем страницу стажировки
        response = requests.get(url)
        response.raise_for_status()

        # Парсим страницу с помощью BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Ищем и извлекаем текст для каждого блока
        task_text = soup.find('div', {'field': 'tn_text_1671359125538'}).text.strip() if soup.find('div', {'field': 'tn_text_1671359125538'}) else "Не найдено"

        purpose_text = soup.find('div', {'field': 'tn_text_1671359125547'}).text.strip() if soup.find('div', {'field': 'tn_text_1671359125547'}) else "Не найдено"

        input_data_text = soup.find('div', {'field': 'tn_text_1671359125559'}).text.strip() if soup.find('div', {'field': 'tn_text_1671359125559'}) else "Не найдено"

        output_data_text = soup.find('div', {'field': 'tn_text_1671359125568'}).text.strip() if soup.find('div', {'field': 'tn_text_1671359125568'}) else "Не найдено"

        # Формируем описание
        details = (f"Задача: {task_text}\n"
                   f"Для чего: {purpose_text}\n"
                   f"Вводные данные: {input_data_text}\n"
                   f"Выходные данные: {output_data_text}")
        return details

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке страницы {url}: {e}")
        return "Ошибка загрузки данных"

**ЗАГРУЖАЕМ ТАБЛИЦУ СТАЖИРОВОК**

In [ ]:
# Загрузка данных
internships_df = load_data()

# Просмотр таблицы данных
internships_df

,URL,Title
0,https://ai-hunter.ru/otp,"Система распознавания документов, компания ОТР..."
1,https://ai-hunter.ru/goodgame,"Рекомендательная система, компания Goodgame, с..."
2,https://ai-hunter.ru/call,Система обработки телефонных аудиозаписей call...
3,https://neural-university.ru/onlinestore,"Рекомендательная система интернет магазина, ст..."
4,https://neural-university.ru/pts,Нейронная сеть для определения веса детали и V...
...,...,...
69,https://ai-hunter.ru/kvanttelecomgpt,Создание нейро-консультанта на основе внутренн...
70,https://ai-hunter.ru/planetazdor,Создание нейро-менеджера чата для ответов на в...
71,https://ai-hunter.ru/sammit,Создание нейро-менеджера чата для внутренней п...
72,https://ai-hunter.ru/icompany,Создание нейро-рекрутёра для подбора IT-специа...


**ПОДГОТОВКА СПИСКА СТАЖИРОВОК**

In [ ]:
# Загрузка данных
internships_df = load_data()

# Исключаем строки с пустыми значениями, "AI", и "GPT"
filtered_internships_df = internships_df[
    (internships_df['Title'].str.strip() != '') &
    (internships_df['Title'] != 'AI') &
    (internships_df['Title'] != 'GPT')
]

# Создаем список с названиями стажировок и ссылками
internships_list = '\n'.join(filtered_internships_df.apply(lambda row: f"{row['Title']}: {row['URL']}", axis=1))
print(internships_list)

Система распознавания документов, компания ОТР 2000, стажировка, AI, CV, портфолио, ленд: https://ai-hunter.ru/otp
Рекомендательная система, компания Goodgame, стажировка, AI, портфолио, ленд: https://ai-hunter.ru/goodgame
Система обработки телефонных аудиозаписей call-центра, компания МедиаБайер, стажировка, AI, портфолио, ленд: https://ai-hunter.ru/call
Рекомендательная система интернет магазина, стажировка, AI, портфолио, ленд: https://neural-university.ru/onlinestore
Нейронная сеть для определения веса детали и VIN
номера ПТС, компания EuroAuto, стажировка, AI, CV, портфолио, ленд: https://neural-university.ru/pts
обнаружение людей, акул и нефтяных пятен с квадрокоптера, компания Дрон Солюшнс, стажировка, AI, CV, портфолио, ленд: https://ai-hunter.ru/quadrocopter
Оценка коммерческой недвижимости в Москве, компания Гринэкспертиза, стажировка, AI, портфолио, ленд: https://ai-hunter.ru/structure
Определение глаукомы по снимку глаза, компания Intemsys, стажировка, медицина, AI, CV, пор

**ВВОДИМ ПРИБЛИЗИТЕЛЬНОЕ НАЗВАНИЕ ПРОШЕДШИХ СТАЖИРОВОК**

In [ ]:
# Вызов функции для получения пользовательского ввода
user_titles = get_user_input()

Введите названия стажировок через запятую (или оставьте пустым, если стажировки не проходили): Распознавание лиц МИЭТ, Задачи NER РусЭкспресс


**ПРОМТЫ ДЛЯ ПОИСКА РЕЛЕВАНТНЫХ СТАЖИРОВОК**

In [ ]:
# Промт для GPT
prompt = """Ты опытный специалист по подбору персонала. Твоя задача - проанализировать названия стажировок, введенные кандидатом,
и сравнить их со списком доступных стажировок. Верни те стажировки из списка, которые максимально соответствуют введенным названиям.
Стажировки пиши один в один как они прописаны в списке. От себя ничего не придумывай. Если подходящих стажировок нет,
напиши "Подходящих стажировок не найдено"."""

# Пример запроса для GPT
query_template = """
Введенные стажировки кандидата: {user_titles}.
Сравни их со следующим списком стажировок и выбери релевантные:
{internships_list}
"""

**НАХОДИМ РЕЛЕВАНТНЫЕ СТАЖИРОВКИ И ДОБАВЛЯЕМ В СУММАРИЗИРОВАННЫЙ ДИАЛОГ**

In [ ]:
# Вызов функции для генерации релевантных стажировок
relevant_internships = generate_relevant_internships(user_titles, internships_list)

# Если релевантные стажировки найдены, добавляем их в диалог
if "Подходящих стажировок не найдено" not in relevant_internships:
    summarized_dialog += "\nПодобранные релевантные стажировки:\n"
    summarized_dialog += relevant_internships
else:
    summarized_dialog += "\nПодходящих стажировок не найдено.\n"

print(summarized_dialog)

Позиция: AI/ML Разработчик

Завершенный курс: Data Science, нейронные сети, машинное обучение и искусственный интеллект
Навыки (стэк): 
Компьютерное зрение: Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, ImageEnhance, Tesseract, PyTesseract, Stable Diffusion, easyocr, pdf2image, TerraYoloV5, SAM, LangSAM, GroundingDINO, Roboflow, CVAT, Ultralytics
Работа с текстами: NLTK, SpaCy, Transformers (Hugging Face), Langchain, Tiktoken, PyDub, SpeechRecognition, Whisper, Bert, GPT, T5
Машинное обучение и глубокое обучение: TensorFlow, Keras, PyTorch (Torch), Scikit-learn, AutoKeras, H2O.ai, TPOT, Transformers (Hugging Face), Horovod, Stable Diffusion, Diffusers
Анализ данных и визуализация: Pandas, Numpy, Matplotlib, Seaborn, Prophet, AutoTS, MoviePy, VidGear, SciPy
Разработка веб-приложений: Flask, FastAPI, Uvicorn, Gunicorn, Django
Системное администрирование и DevOps: Docker, Docker Compose, Nginx, OS, Shutil, Glob, VSCode, dotenv, python-telegram-bot, pygit2
Обработка звука 

### **4. Блок резюме**

In [ ]:
#@title Сервисные функции четвертого блока

# Функция для запроса резюме у пользователя
def resume_dialog() -> str:
    print(format_text("Пожалуйста, пришлите ваше резюме в текстовом виде."), '\n')
    resume = input("Ваше резюме: ")
    return resume

**ЗАГРУЖАЕМ СТАРОЕ РЕЗЮМЕ ИЛИ ТЕКСТОВУЮ ИНФОРМАЦИЮ ПО ОПЫТУ РАБОТЫ**

In [ ]:
# Вызов функции диалога для резюме
user_resume = resume_dialog()
summarized_dialog += f"\nРезюме пользователя:\n{user_resume}\n"

Пожалуйста, пришлите ваше резюме в текстовом виде. 

Ваше резюме: Петров Петр Петрович  29 лет (родился 05 мая 1995), высшее образование Челябинск, ул. Квартальная, д. 1 Email: ppp1010@gmail.com Телефон: +7(789)323-23-45 Гражданство: РФ Питон-разработчик Полная занятость 100 000 ₽ Опыт работы 1 год Сентябрь 2018 – август 2019 1 год Питон разработчик SkyEnglish, Москва Известная онлайн-платформа для изучения английского языка, более 1 млн. пользователей по всему миру Обязанности: - Связал данные мобильного приложения с данными сайта. - Написал код с функционалом чтения, записи и загрузки данных с мобильного и веб приложения в базу. - Создал 2 графика для анализа данных с фильтрами по датам (ежедневно, еженедельно, ежемесячно и ежегодно). Графики включали в себя общие данные о записях пользователя, воспроизведенные с помощью line chart, bar chart & pie chart. Каждая точка на графике может быть детально изучена. - Оптимизировал более 20000 строк кода: Python - 7500, Java for Android - 130

**ВЫВОДИМ СУММАРИЗИРОВАННЫЙ ФАЙЛ С ДАННЫМИ КАНДИДАТА**

In [ ]:
print(summarized_dialog)

Позиция: AI/ML Разработчик

Завершенный курс: Data Science, нейронные сети, машинное обучение и искусственный интеллект
Навыки (стэк): 
Компьютерное зрение: Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, ImageEnhance, Tesseract, PyTesseract, Stable Diffusion, easyocr, pdf2image, TerraYoloV5, SAM, LangSAM, GroundingDINO, Roboflow, CVAT, Ultralytics
Работа с текстами: NLTK, SpaCy, Transformers (Hugging Face), Langchain, Tiktoken, PyDub, SpeechRecognition, Whisper, Bert, GPT, T5
Машинное обучение и глубокое обучение: TensorFlow, Keras, PyTorch (Torch), Scikit-learn, AutoKeras, H2O.ai, TPOT, Transformers (Hugging Face), Horovod, Stable Diffusion, Diffusers
Анализ данных и визуализация: Pandas, Numpy, Matplotlib, Seaborn, Prophet, AutoTS, MoviePy, VidGear, SciPy
Разработка веб-приложений: Flask, FastAPI, Uvicorn, Gunicorn, Django
Системное администрирование и DevOps: Docker, Docker Compose, Nginx, OS, Shutil, Glob, VSCode, dotenv, python-telegram-bot, pygit2
Обработка звука 

### **5. Блок создания шаблонного резюме**

In [ ]:
#@title Сервисные функции шестого блока

# Функция генерации резюме
def generate_resume(prompt, query):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        temperature=0,
        max_tokens=4000
    )

    return response['choices'][0]['message']['content']

**ПРОМТЫ ДЛЯ ГЕНЕРАЦИИ РЕЗЮМЕ**

In [ ]:
# Промт для генерации резюме
resume_prompt = """
Ты опытный HR-специалист и специалист в области компьютерых технологий и программирования. На основании предоставленной информации составь шаблонное
 резюме исключительно по шаблонной структуре. Опыт работы обязательно начни с блока Университета искусственного интеллекта как в шаблоне ниже,
 если каких-то блоков нет, просто пиши нет данных, обязательно используй все данные по образованию в Университете искусственного интеллекта.
 Обязательно добавь весь блок Навыки (стэк) именно под стажировками в структуре резюме.
 Согласно шаблону опиши все прошедшие кандидатом стажировки в полном объеме, добавив краткое описание стажировки вместо строчек:
 задача, для чего, вводные данные, выходные данные. Обязательно учти пожелания и приоритеты кандидата по итогам его ответов.
Пример шаблонного резюме:

1. ЛИЧНЫЕ ДАННЫЕ:
Имя: Иванов Иван Иванович
Адрес: 111111, г. Москва, ул. Якорная, д. 1, к. 1, кв. 1
Email: IvanovIvan@gmail.com
Телефон: +1(111)111-11-11

2. ДОЛЖНОСТЬ:
AI/ML Разработчик
Зарплата: 100 000 рублей в месяц
Формат работы: Удаленная работа

3. ОПЫТ РАБОТЫ:
УНИВЕРСИТЕТ ИСКУССТВЕННОГО ИНТЕЛЛЕКТА | 11.2023-06.2024
Прохождение курса: «Data Science, нейронные сети, машинное обучение и искусственный интеллект»

Стажировки:
Система выделения нистагма из фонового шума при быстром перемещении взора пациента, компания РУДН, ТОП компания, стажировка, медицина, AI, CV, портфолио, ленд
Краткое описание: Стажировка по разработке системы для выделения типовых следящих движений глаза. Цель проекта — помочь специалистам в диагностике, сегментируя начало и конец сигналов. Используемые данные включают временные ряды, что позволяет точнее распознавать и обрабатывать полезные сигналы.
Ссылка на лэнд: https://ai-hunter.ru/nystagmus

Разработка рекомендательной системы для интернет-магазина, компания ВанДеливери, стажировка, AI, портфолио, ленд
Краткое описание: Стажировка по созданию автоматизированной рекомендательной системы для платформы One Delivery. Цель проекта — увеличение величины чека за счет эффективных рекомендаций и автоматизация процесса формирования рекомендаций. Входные данные включают истории покупок через приложение, что позволяет улучшить эффективность и актуальность рекомендаций.
Ссылка на лэнд: https://ai-hunter.ru/onedelivery

Обладаю компетенциями в решении всех основных задач АИ.
Навыки (стэк):
Компьютерное зрение: Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, ImageEnhance, Tesseract, PyTesseract, Stable Diffusion, easyocr, pdf2image, TerraYoloV5, SAM, LangSAM, GroundingDINO, Roboflow, CVAT, Ultralytics
Работа с текстами: NLTK, SpaCy, Transformers (Hugging Face), Langchain, Tiktoken, PyDub, SpeechRecognition, Whisper, Bert, GPT, T5
Машинное обучение и глубокое обучение: TensorFlow, Keras, PyTorch (Torch), Scikit-learn, AutoKeras, H2O.ai, TPOT, Transformers (Hugging Face), Horovod, Stable Diffusion, Diffusers
Анализ данных и визуализация: Pandas, Numpy, Matplotlib, Seaborn, Prophet, AutoTS, MoviePy, VidGear, SciPy
Разработка веб-приложений: Flask, FastAPI, Uvicorn, Gunicorn, Django
Системное администрирование и DevOps: Docker, Docker Compose, Nginx, OS, Shutil, Glob, VSCode, dotenv, python-telegram-bot, pygit2
Обработка звука и речи: SpeechRecognition, PyDub, gTTS, TTS, LibROSA, Jiwer, PyAudio, Whisper, Google Text-to-Speech
Работа с данными и API: Requests, FAISS, OpenAI, Gspread, Google API Client, Telegram API, OpenAI API
Мультимедийная обработка: MoviePy, PIL, pdf2image, VidGear
Прочие навыки и инструменты: Google Colaboratory, Getpass, ImageEnhance, PIL, VSCode, Telegram.ext, dotenv

ООО «ИНТУИЦИЯ» | 02.2022-12.2023 гг.
Руководитель проектов
- Реализовал пилотное внедрение геоинформационной системы в рабочие процессы предприятия.

АО "ИНСТИТУТ РАЗВИТИЯ" | 05.2018-01.2022 гг.
Разработчик
- Разрабатывал проекты для повышения эффективности рабочих процессов организации.

4. ОБРАЗОВАНИЕ:
Первая национальная школа телевидения | 2010
- Диплом режиссера телевизионных программ

Московский Государственный Университет | 2009
- Диплом Бакалавра технических наук

5. ДОПОЛНИТЕЛЬНЫЕ НАВЫКИ:
- Умение работать в команде на результат.
- Глубокое понимание работы нейронных сетей и их интеграции в рабочие процессы.
- Разработка и внедрение программных решений на Python, SQL, HTML, CSS, JavaScript и др.
- Опыт управления проектами и автоматизации процессов.

6. ДОПОЛНИТЕЛЬНОЕ ОБУЧЕНИЕ И СЕРТИФИКАЦИЯ:
- Сертификат по управлению проектами, PMP | 2021
- Онлайн курс по машинному обучению, Coursera | 2020

7. НАГРАДЫ И ДОСТИЖЕНИЯ:
- Победитель хакатона «AI for Good» (2022)
- Второе место на хакатоне «Machine Learning Challenge» (2021)
- Призер хакатона «Data Science Hackathon» (2020)

8. ЯЗЫКИ:
- Русский: Родной
- Английский: Свободное владение
- Немецкий: Базовый уровень
"""

# Генерация запроса для GPT
query = f"""Сформируйте резюме по шаблонной структуре, ничего не придумывай от себя, обязательно укажи зарплату,
к каждой стажировке при наличие обязательно добавь краткое описание в двух предложениях на основании имеющейся информации,
укажи все места работы кандидата, строго соблюдай структуру шаблонного резюме.
Обязательно впиши в резюме все Навыки (стэк) из файла диалога с расшифровкой по областям применения после и вставь после блока стажировок как в шаблоне.
Резюме создай исключительно на основании следующей информации:\n\n{summarized_dialog}"""

**ГЕНЕРАЦИЯ НОВОГО РЕЗЮМЕ**

In [ ]:
# Вызов функции для генерации резюме
generated_resume = generate_resume(resume_prompt, query)

# Вывод сгенерированного резюме
print(f"\nСформированное резюме:\n{generated_resume}\n")


Сформированное резюме:
1. ЛИЧНЫЕ ДАННЫЕ:
Имя: Петров Петр Петрович  
Адрес: Челябинск, ул. Квартальная, д. 1  
Email: ppp1010@gmail.com  
Телефон: +7(789)323-23-45  
Гражданство: РФ  

2. ДОЛЖНОСТЬ:
AI/ML Разработчик  
Зарплата: 100 000 рублей в месяц  
Формат работы: Офис  

3. ОПЫТ РАБОТЫ:
УНИВЕРСИТЕТ ИСКУССТВЕННОГО ИНТЕЛЛЕКТА | 09.2023-05.2024  
Прохождение курса: «Data Science, нейронные сети, машинное обучение и искусственный интеллект»  

Стажировки:  
Разработка системы распознавания лиц сотрудников с целью перемещения по объектам предприятия, компания МИЭТ, стажировка, AI, CV, портфолио, ленд  
Краткое описание: Стажировка по созданию и интеграции нейросетевой модели для распознавания лиц сотрудников. Разработанная система может быть интегрирована в систему видеонаблюдения для контроля перемещения сотрудников и восстановления последовательности событий.  
Ссылка на лэнд: https://ai-hunter.ru/recognition  

Разработка модели для задачи NER обработки тестовых данных и нахождения

### **6. Блок подбора вакансий**

In [ ]:
#@title Сервисные функции шестого блока

# Промт для генерации профайла из резюме
profile_generation_prompt = """
Ты опытный ассистент по написанию профайлов вакансий. На основании предоставленного текста резюме составь шаблонный профайл исключительно по следующей структуре:

Пример шаблонного профайла:
1. Позиция: AI/ML Разработчик
2. Локация: Москва
3. Зарплата: 100 000 рублей в месяц
4. Формат работы: Офис
5. Задачи: CV - распознование лиц.
6. Стэк (библиотеки/фреймворки): Google Colaboratory, Python, Numpy, Matplotlib, TensorFlow, Keras, PyTorch (Torch), Scikit-learn, NLTK, SpaCy, Transformers (Hugging Face), AutoKeras, H2O.ai, TPOT, Pandas, Seaborn, Prophet, AutoTS, Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, Flask, FastAPI, Uvicorn, Tesseract, PyTesseract, Stable Diffusion, SpeechRecognition, PyDub, gTTS, TTS, Langchain, OS, Shutil, Glob, Nginx, Gunicorn, Docker, Docker Compose, TensorRT, Horovod, MoviePy, Requests, FAISS, OpenAI, Tiktoken, Getpass, ImageEnhance, PIL, SciPy, VidGear, LibROSA, Jiwer, PyAudio, pygit2, easyocr, pdf2image, pyspellchecker, Telegram.ext, dotenv, TerraYoloV5, VSCode, python-telegram-bot, SAM, LangSAM, GroundingDINO, Whisper, Roboflow, CVAT, Ultralytics, Diffusers, Google Text-to-Speech, Transformer models (embeddings, attention mechanisms), Bert, GPT, T5

Если в резюме данных нет, так и пиши: нет данных. В пункт: 6. Стэк - (библиотеки/фреймворки) - добавляй исключительно библиотеки или фреймворки, с которыми работал кандидат. В пункт: 5. Задачи - дополнительно внеси все интересующие кандидата проекты и любые желания и приоритеты. Формируй профайл четко по заданным правилам, исключительно на основании предоставленного резюме:
"""

# Функция для загрузки данных из Google Sheets
def load_data():
    # Открываем Google Sheet по URL
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1sXOm9lnWO_zMLvAxV8ORxlnrYOjnSSV9X-SyTqZUVoQ/edit?usp=sharing')
    # Открываем нужную вкладку
    worksheet = sh.worksheet('05.08.2024')

    # Считываем все данные с листа
    rows = worksheet.get_all_values()

    # Переименовываем первую строку в заголовки столбцов
    headers = rows[1]  # Вторая строка (индекс 1) содержит заголовки
    data = rows[2:]    # Данные начинаются с третьей строки (индекс 2)

    # Преобразуем данные в DataFrame
    dataframe = pd.DataFrame(data, columns=headers)
    return dataframe

# Функция для парсинга вакансии HH и сохранения данных в JSON
def parse_and_save_vacancy_with_excel_data(url, file_name, excel_data):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
        }

        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Парсинг страницы с помощью BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Поиск и извлечение данных из JSON в <script type="application/ld+json">
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if script_tag:
            vacancy_data = json.loads(script_tag.string)

            # Добавляем данные из Excel в JSON
            vacancy_data['excel_data'] = excel_data

            # Сохранение данных в JSON файл
            with open(os.path.join('/content/all_vacancies', f'{file_name}.json'), 'w', encoding='utf-8') as f:
                json.dump(vacancy_data, f, ensure_ascii=False, indent=4)
            print(f"Вакансия сохранена в файл: {file_name}.json")
        else:
            print(f"JSON-данные не найдены на странице: {url}")
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке страницы {url}: {e}")

# Функция для использования GPT для извлечения стека технологий
def extract_stack_with_gpt(description):
    prompt = "Вытащи все библиотеки, фреймворки и технологии на русском и на английском, упомянутые в следующем описании вакансии, не используй \n, просто перечисли через запятую."
    query = f"Описание вакансии: {description}"

    # Используем функцию generate_resume для вызова GPT
    stack = generate_resume(prompt, query)

    return stack

# Функция генерации профиля вакансии
def generate_job_profile(vacancy_data):
    """
    Преобразует данные вакансии в шаблонный профайл по 6 ключевым позициям.
    """
    # Парсим описание вакансии с помощью BeautifulSoup
    soup = BeautifulSoup(vacancy_data.get("description", ""), "html.parser")

    # Извлекаем задачи и функции
    tasks = " ".join([task.get_text(strip=True) for task in soup.find_all('li')]).lower()

    # Используем GPT для извлечения стека технологий
    stack = extract_stack_with_gpt(tasks)

    # Формируем профайл
    job_profile = {
        "Позиция": vacancy_data.get("title", "Не указано"),
        "Локация": vacancy_data.get("jobLocation", {}).get("address", {}).get("addressLocality", "Не указано"),
        "Зарплата": vacancy_data.get("excel_data", {}).get("ЗП", "Не указано"),
        "Формат работы": vacancy_data.get("excel_data", {}).get("Формат работы", "Не указано"),
        "Задачи": tasks if tasks else "Не указано",
        "Стэк (библиотеки/фреймворки)": stack if stack else "Не указано"
    }

    return job_profile

# Функция генерации резюме с использованием GPT
def generate_resume(prompt, query):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        temperature=0,
        max_tokens=3000
    )

    return response['choices'][0]['message']['content']

def process_vacancies(vacancy_dir, output_dir):
    """
    Обрабатывает все вакансии в указанной директории, преобразует их в профайлы и сохраняет в отдельную папку.
    """
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(vacancy_dir):
        if filename.endswith('.json'):
            with open(os.path.join(vacancy_dir, filename), 'r', encoding='utf-8') as file:
                vacancy_data = json.load(file)
                profile = generate_job_profile(vacancy_data)

                # Сохранение профайла в отдельный файл
                profile_filename = os.path.join(output_dir, f"profile_{filename}")
                with open(profile_filename, 'w', encoding='utf-8') as profile_file:
                    json.dump(profile, profile_file, ensure_ascii=False, indent=4)

    print(f"Все профайлы сохранены в папке {output_dir}")

# Функция для генерации профайла из резюме
def generate_profile_from_resume(resume_text, prompt=profile_generation_prompt):
    query = f"\n\nРезюме пользователя:\n{resume_text}"

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        temperature=0,
        max_tokens=512
    )

    return response['choices'][0]['message']['content']

# Оценка вакансий от 1 до 10 баллов (перевёрнутая шкала)
def calculate_score(similarity_score, min_score=0.1, max_score=0.5):
    # Нормализация значения в диапазон [1, 10]
    normalized_score = 10 * (1 - (similarity_score - min_score) / (max_score - min_score))
    return round(normalized_score, 1)

def refine_search_with_gpt(profile, initial_results):
    refined_results = []

    for doc, score in initial_results:
        # Создаем запрос для GPT
        prompt = f"""
        Кандидат имеет следующие задачи и навыки:
        {profile}

        Описание вакансии:
        {doc.page_content}

        Оцените соответствие профиля кандидата задачам и навыкам вакансии. Максимально критично оцените кандидата: если у него отсутствуют важные навыки или опыт, указанные в вакансии, это должно существенно снизить его оценку. В случае отсутствия необходимых навыков или опыта, кандидат не подходит. Дайте итоговую оценку от 0 до 10 баллов, где 10 - полное соответствие кандидата вакансии. Укажите эту оценку в конце ответа и краткое заключение.
        """

        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Вы критически настроенный специалист по подбору персонала."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,  # Уменьшаем температуру для более предсказуемых ответов
            max_tokens=1000
        )

        # Извлекаем текст ответа от GPT
        gpt_response_text = response['choices'][0]['message']['content']

        # Ищем числовую оценку в конце текста с помощью регулярного выражения
        match = re.search(r'Итоговая оценка:.*?(\d+(\.\d+)?)', gpt_response_text)
        if match:
            gpt_score = float(match.group(1))
        else:
            gpt_score = 0.0  # если оценка не найдена, назначаем 0

        # Извлечение краткого заключения или генерация альтернативного сообщения
        conclusion_match = re.search(r'Итоговая оценка:.*\n(.*)', gpt_response_text, re.DOTALL)
        if conclusion_match and conclusion_match.group(1).strip():
            gpt_conclusion = conclusion_match.group(1).strip()
        else:
            gpt_conclusion = "Кандидат имеет некоторые соответствия с вакансией, но отсутствуют ключевые навыки или опыт, что снижает его соответствие."

        refined_results.append((doc, gpt_score, gpt_conclusion))

    # Сортировка и отбор лучших 5 результатов
    refined_results.sort(key=lambda x: x[1], reverse=True)
    return refined_results[:5]

def select_relevant_vacancy(profile, embeddings, db_path='/content/profiles_faiss_index_extracted', num_vacancies=5, k=20):
    # Загрузка индексной базы FAISS для профилей
    db = FAISS.load_local(db_path, embeddings)

    # Поиск первых k наиболее релевантных вакансий с оценкой по схожести
    initial_results = db.similarity_search_with_score(profile, k=k)

    # Применение функции для уточнения поиска с использованием GPT
    final_results = refine_search_with_gpt(profile, initial_results)

    # Функция для форматирования текста
    def format_text(text, width=120):
        return '\n'.join(textwrap.fill(line, width) for line in str(text).split('\n'))

    # Вывод num_vacancies самых релевантных вакансий (без оценок и кратких заключений)
    for i, (doc, _, _) in enumerate(final_results[:num_vacancies]):
        vacancy_filename = doc.metadata['source']
        formatted_description = format_text(doc.page_content, width=120)

        print(f"{i+1}. Вакансия: {vacancy_filename}")
        print(f"Полное описание:\n{formatted_description}\n")
        print("="*50)

    # Ввод пользователем номера вакансии
    selected_index = int(input(f"Введите номер вакансии, которую хотите выбрать (1-{num_vacancies}): ")) - 1

    # Сохранение выбранной вакансии
    selected_vacancy = final_results[selected_index][0]

    # Вывод выбранной вакансии
    print(f"\nВы выбрали вакансию: {selected_vacancy.metadata['source']}")
    formatted_selected_description = format_text(selected_vacancy.page_content, width=120)
    print(f"Полное описание вакансии:\n{formatted_selected_description}")

    # Возвращаем выбранную вакансию
    return selected_vacancy

# Скачиваем файл
url = 'https://drive.google.com/uc?id=1dBbM79TFQYchIb8NKA-skLAlbfrS_gew'
output = 'vacancies.zip'
gdown.download(url, output, quiet=False)

# Распаковываем ZIP-архив в корневую папку content
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('/content')

Downloading...
From: https://drive.google.com/uc?id=1dBbM79TFQYchIb8NKA-skLAlbfrS_gew
To: /content/vacancies.zip
100%|██████████| 174k/174k [00:00<00:00, 49.9MB/s]


**ЗАГРУЖАЕМ ДАННЫЕ ИЗ EXCEL ТАБЛИЦЫ**

In [ ]:
# Загрузка данных
vacancies_df = load_data()

# Просмотр таблицы данных
vacancies_df

,Позиция,Компания,Локация,ЗП,График работы,Формат работы,Ссылка
0,Data Scientist,Сбер,Москва,не указано,5/2,офис/гибрид,https://rabota.sber.ru/search/4323750/
1,Junior Quantitative Researcher,ООО Актив Матрикс,Москва,150 000 - 250 000 ₽,5/2,офис/гибрид или удаленная работа,https://hh.ru/vacancy/103079830
2,Исследователь/Research Scientist,Криптонит,Москва,не указано,5/2,офис/гибрид или удаленная работа,https://hh.ru/vacancy/102620025
3,Стажер-разработчик Python,Додо Пицца,Ростов-на-Дону,от 100 000 ₽,5/2,офис/гибрид,https://hh.ru/vacancy/104634531
4,Data Engineer (Smart CAPEX),Beeline,Москва,не указано,5/2,офис,https://job.beeline.ru/jobs/moskva/data-engine...
...,...,...,...,...,...,...,...
164,Data Engineer,Мегафон,Москва,не указано,5/2,офис/гибрид или удаленная работа,https://career.habr.com/vacancies/1000143688
165,Machine Learning Engineer : AI Shopping Assistant,Constructor,"Белград, Сербия",90-110k USD,5/2,удаленная работа,https://apply.workable.com/constructor-1/j/CC1...
166,Python Developer,АО Гринатом,Москва,не указано,5/2,удаленная работа,https://hh.ru/vacancy/99009796
167,AI Product/Project manager по внедрению технол...,Сбер,Сочи,не указано,5/2,офис/гибрид,https://rabota.sber.ru/search/4299923/


**ПАРСИМ ВСЕ ССЫЛКИ НА HEAD HUNTER**

In [ ]:
# Создаем папку для сохранения всех вакансий
output_dir = "/content/all_vacancies"
os.makedirs(output_dir, exist_ok=True)

# Загружаем данные из Google Sheets
df = load_data()

# Фильтрация ссылок на HeadHunter
hh_links = df[df['Ссылка'].str.contains('hh.ru')]

# Цикл по всем ссылкам и парсинг данных
for index, row in hh_links.iterrows():
    url = row['Ссылка']
    file_name = f"vacancy_{index + 1}"  # +1, чтобы соответствовать строкам в Excel
    excel_data = row.to_dict()  # Преобразуем строку из Excel в словарь
    parse_and_save_vacancy_with_excel_data(url, file_name, excel_data)
    time.sleep(2)  # Задержка в 2 секунды между запросами

Вакансия сохранена в файл: vacancy_2.json
Вакансия сохранена в файл: vacancy_3.json
Вакансия сохранена в файл: vacancy_4.json
Вакансия сохранена в файл: vacancy_8.json
Вакансия сохранена в файл: vacancy_9.json
Вакансия сохранена в файл: vacancy_10.json
Вакансия сохранена в файл: vacancy_11.json
Вакансия сохранена в файл: vacancy_15.json
Вакансия сохранена в файл: vacancy_16.json
Вакансия сохранена в файл: vacancy_17.json
Вакансия сохранена в файл: vacancy_20.json
Вакансия сохранена в файл: vacancy_23.json
Вакансия сохранена в файл: vacancy_24.json
Вакансия сохранена в файл: vacancy_30.json
Вакансия сохранена в файл: vacancy_31.json
Вакансия сохранена в файл: vacancy_32.json
Вакансия сохранена в файл: vacancy_36.json
Вакансия сохранена в файл: vacancy_37.json
Вакансия сохранена в файл: vacancy_39.json
Вакансия сохранена в файл: vacancy_44.json
Вакансия сохранена в файл: vacancy_45.json
Ошибка при загрузке страницы https://hh.ru/vacancy/104469801: 403 Client Error: Forbidden for url: htt

**ВЫВОДИМ ПРИМЕР ОДНОЙ ВАКАНСИИ**

In [ ]:
# Пример вывода информации вакансии из JSON файла
with open('/content/all_vacancies/vacancy_10.json', 'r', encoding='utf-8') as f:
    vacancy_data = json.load(f)

# Пример извлечения и вывода основных данных
title = vacancy_data.get('title')
company = vacancy_data.get('hiringOrganization', {}).get('name')
location = vacancy_data.get('jobLocation', {}).get('address', {}).get('addressLocality')
description = vacancy_data.get('description')

print(f"Вакансия: {title}")
print(f"Компания: {company}")
print(f"Локация: {location}")
print(f"Описание:\n{description}")

Вакансия: ML Engineer/Data Scientist
Компания: Oil and Gas Production Tools
Локация: Санкт-Петербург
Описание:
<p><em>Мы ищем </em><strong><em>ML Engineer / Data Scientist</em></strong><em> для направления разработки систем, направленных на моделирование и оптимизацию процессов добычи нефти и газа.</em></p> <p><strong>Задачи:</strong></p> <ul> <li>Сбор, парсинг, анализ, классификация разрозненных данных.</li> <li>Исследования взаимовлияния, построение матриц корреляций, анализ сырых временных рядов</li> <li>Восстановление временных рядов по косвенным показателям с других временных рядов</li> <li>Также есть задачи на анализ текстов и работа с LLM моделями</li> <li>Анализ предметной области моделирования и оптимизации технологических процессов добычи, первичной переработки и транспорта нефти и газа</li> <li>Проектирование функциональной архитектуры системы, логической модели данных и API интеграции.</li> <li>Оптимизация производительности</li> <li>Взаимодействие с командой разработчиков 

In [ ]:
#@title Переводим все вакансии в профайлы

# Путь к директории с вакансиями
vacancy_dir = '/content/all_vacancies'

# Путь к директории для сохранения профайлов
output_dir = '/content/job_profiles'

# Обрабатываем вакансии и сохраняем профайлы
process_vacancies(vacancy_dir, output_dir)

Все профайлы сохранены в папке /content/job_profiles


**ВЫВОДИМ ПРИМЕР ОДНОГО ПРОФАЙЛА**

In [ ]:
# Путь к файлу
file_path = '/content/job_profiles/profile_vacancy_10.json'

# Открываем и читаем содержимое файла
with open(file_path, 'r', encoding='utf-8') as file:
    profile_data = json.load(file)

# Выводим содержимое на экран
print(json.dumps(profile_data, ensure_ascii=False, indent=4))

{
    "Позиция": "ML Engineer/Data Scientist",
    "Локация": "Санкт-Петербург",
    "Зарплата": "не указано",
    "Формат работы": "удаленная работа",
    "Задачи": "сбор, парсинг, анализ, классификация разрозненных данных. исследования взаимовлияния, построение матриц корреляций, анализ сырых временных рядов восстановление временных рядов по косвенным показателям с других временных рядов также есть задачи на анализ текстов и работа с llm моделями анализ предметной области моделирования и оптимизации технологических процессов добычи, первичной переработки и транспорта нефти и газа проектирование функциональной архитектуры системы, логической модели данных и api интеграции. оптимизация производительности взаимодействие с командой разработчиков и тестировщиков умение аккумулировать данные из различных источников, объединять, приводить к единой дате опыт работы с временными рядами, понимание применяемых методов и опыт работы с соответствующими библиотеками python подбор релевантных и рас

In [ ]:
#@title Переводим все профайлы в индексную базу FAISS

# Путь к папке с файлами профайлов
profiles_folder = '/content/job_profiles/'  # Укажите путь к папке с файлами профайлов

# Чтение всех JSON файлов с профайлами и создание векторов
profile_documents = []
for filename in os.listdir(profiles_folder):
    if filename.endswith('.json'):
        with open(os.path.join(profiles_folder, filename), 'r', encoding='utf-8') as file:
            profile_data = json.load(file)
            # Превращаем каждый профайл в отдельный вектор (один чанк)
            profile_text = " ".join([f"{key}: {value}" for key, value in profile_data.items()])
            profile_documents.append(Document(page_content=profile_text, metadata={"source": filename}))

# Создание индексной базы FAISS
db = FAISS.from_documents(profile_documents, embeddings)

# Сохранение индексной базы
db.save_local('/content/profiles_faiss_index')

# Вывод количества созданных профайлов (вакансий)
print(f"Векторная база данных профайлов успешно создана и сохранена. Всего профайлов: {len(profile_documents)}")

Векторная база данных профайлов успешно создана и сохранена. Всего профайлов: 77


In [ ]:
#@title Сохраняем векторную базу

# Указываем путь к директории, которую нужно сохранить
source_dir = '/content/profiles_faiss_index'

# Архивируем директорию в формат ZIP и сохраняем в /content
shutil.make_archive('/content/profiles_faiss_index', 'zip', source_dir)

print("Архив успешно создан и сохранен в /content/profiles_faiss_index.zip")

Архив успешно создан и сохранен в /content/profiles_faiss_index.zip


In [ ]:
#@title Загружаем векторную базу
# Скачиваем архив с векторной базой
url = 'https://drive.google.com/uc?id=1srye-tne67llJBcP5OTRTQBHK1m3vml9'
output = '/content/profiles_faiss_index.zip'
gdown.download(url, output, quiet=False)

# Указываем папку для распаковки архива
extract_dir = '/content/profiles_faiss_index_extracted'

# Распаковываем архив в указанную папку
shutil.unpack_archive(output, extract_dir)

print(f"Архив распакован в папку: {extract_dir}")

# Загрузка базы FAISS
index = faiss.read_index(f"{extract_dir}/index.faiss")

Downloading...
From: https://drive.google.com/uc?id=1srye-tne67llJBcP5OTRTQBHK1m3vml9
To: /content/profiles_faiss_index.zip
100%|██████████| 460k/460k [00:00<00:00, 85.3MB/s]

Архив распакован в папку: /content/profiles_faiss_index_extracted


**ПЕРЕВОДИМ РЕЗЮМЕ В ПРОФАЙЛ**

In [ ]:
# Промт для генерации профайла из резюме
profile_generation_prompt = """
Ты опытный ассистент по написанию профайлов вакансий. На основании предоставленного текста резюме
составь шаблонный профайл исключительно по следующей структуре:

Пример шаблонного профайла:
1. Позиция: AI/ML Разработчик
2. Локация: Москва
3. Зарплата: 100 000 рублей в месяц
4. Формат работы: Офис
5. Задачи: CV - распознование лиц.
6. Стэк (библиотеки/фреймворки): Google Colaboratory, Python, Numpy, Matplotlib, TensorFlow, Keras, PyTorch (Torch), Scikit-learn, NLTK, SpaCy, Transformers (Hugging Face), AutoKeras, H2O.ai, TPOT, Pandas, Seaborn, Prophet, AutoTS, Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, Flask, FastAPI, Uvicorn, Tesseract, PyTesseract, Stable Diffusion, SpeechRecognition, PyDub, gTTS, TTS, Langchain, OS, Shutil, Glob, Nginx, Gunicorn, Docker, Docker Compose, TensorRT, Horovod, MoviePy, Requests, FAISS, OpenAI, Tiktoken, Getpass, ImageEnhance, PIL, SciPy, VidGear, LibROSA, Jiwer, PyAudio, pygit2, easyocr, pdf2image, pyspellchecker, Telegram.ext, dotenv, TerraYoloV5, VSCode, python-telegram-bot, SAM, LangSAM, GroundingDINO, Whisper, Roboflow, CVAT, Ultralytics, Diffusers, Google Text-to-Speech, Transformer models (embeddings, attention mechanisms), Bert, GPT, T5

Если в резюме данных нет, так и пиши: нет данных.
В пункт: 6. Стэк - (библиотеки/фреймворки) - добавляй исключительно библиотеки или фреймворки, с которыми работал кандидат.
В пункт: 5. Задачи - дополнительно внеси все интересующие кандидата проекты и любые желания и приоритеты.
Формируй профайл четко по заданным правилам, исключительно на основании предоставленного резюме:
"""

In [ ]:
# Пример использования функции с текстом сгенерированного резюме
profile = generate_profile_from_resume(generated_resume)
print(profile)

1. Позиция: AI/ML Разработчик  
2. Локация: Челябинск  
3. Зарплата: 100 000 рублей в месяц  
4. Формат работы: Офис  
5. Задачи: Разработка системы распознавания лиц, автоматизация обработки тендерных документов, создание и интеграция нейросетевых моделей, работа с компьютерным зрением и обработкой текстов.  
6. Стэк (библиотеки/фреймворки): Detectron2, DeepLab, YOLOv5, YOLOv8, OpenCV, DeepSORT, Pillow, ImageEnhance, Tesseract, PyTesseract, Stable Diffusion, easyocr, pdf2image, TerraYoloV5, SAM, LangSAM, GroundingDINO, Roboflow, CVAT, Ultralytics, NLTK, SpaCy, Transformers (Hugging Face), Langchain, Tiktoken, PyDub, SpeechRecognition, Whisper, Bert, GPT, T5, TensorFlow, Keras, PyTorch (Torch), Scikit-learn, AutoKeras, H2O.ai, TPOT, Horovod, Pandas, Numpy, Matplotlib, Seaborn, Prophet, AutoTS, MoviePy, VidGear, SciPy, Flask, FastAPI, Uvicorn, Gunicorn, Django, Docker, Docker Compose, Nginx, OS, Shutil, Glob, VSCode, dotenv, python-telegram-bot, pygit2, Requests, FAISS, OpenAI, Gspread,

**ПОДБИРАЕМ 5 РЕЛЕВАНТНЫХ ВАКАНСИЙ**

In [ ]:
selected_vacancy = select_relevant_vacancy(profile, embeddings)

1. Вакансия: profile_vacancy_141.json
Полное описание:
Позиция: Data Scientist Компания: РУССКАЯ МОРСКАЯ КОМАНДА Локация: Санкт-Петербург Зарплата: не указано Формат работы:
офис Задачи: Разработка, тестирование и внедрение алгоритмов машинного обучения (CV, NLP) в соответствии с поставленными
задачами Участие в разработке / проверке гипотез, интерпретации результатов и оценке качества разработанных моделей
Подготовка прототипов и проектирование архитектуры решений на базе искусственного интеллекта Исследование новейших ML-
технологий, предложение идей для улучшения качества моделей Высшее образование Английский язык на уровне В1-Intermediate
Хорошие знания и опыт практического применения теории вероятностей и математической статистики, методов машинного
обучения и искусственного интеллекта Знание Python, ML/DL, основных CV и NLP инструментов (трансформенные модели, модели
на основе сверток; pyTorch/Tensorflow; numpy, pandas, OpenCV и др.) Опыт обучения и внедрения моделей ML/DL (в
час

### **7. Блок корректировки резюме**

In [ ]:
#@title Сервисные функции восьмого блока
# Функция для корректировки резюме под выбранную вакансию
def adjust_resume(resume, job_description):
    # Формируем сообщение
    user_message = resume_job_message_template.format(resume=resume, job_description=job_description)

    messages = [
        {"role": "system", "content": resume_adapt_prompt},
        {"role": "user", "content": user_message}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        max_tokens=4000
    )

    return response['choices'][0]['message']['content']

**ПРОМТЫ КОРРЕКТИРОВКИ РЕЗЮМЕ ПОД ВЫБРАННУЮ ВАКАНСИЮ**

In [ ]:
 # Промт для корректировки резюме под выбранную вакансию
resume_adapt_prompt = """Ты опытный HR-специалист. Проанализируй описание вакансии и резюме и создай дополнительный пункт после пункта ДОЛЖНОСТЬ.
Этот будет пункт 3. КЛЮЧЕВЫЕ НАВЫКИ - куда перенеси и распиши все навыки кандидата которые совпадают с необходимыми навыками и умениями в вакансии,
чтобы быстро можно было понять, что знает и умеет кандидат. Измени должность на ту, которую ищет соискатель,
зарплату и формат работы если они указаны в вакансии. Все остальные блоки должны остаться не изменными,
поменяется только нумерация. Обязательно оставь полный стэк умений после блока стажировок."""

# Шаблон для формирования сообщения с резюме и описанием вакансии
resume_job_message_template = "Резюме: {resume}\n\nОписание вакансии: {job_description}\n\nПодкорректируй резюме добавив пункт 3. КЛЮЧЕВЫЕ НАВЫКИ - куда перенеси и распиши все навыки и умения кандидата, которые совпадают с необходимыми навыками и умениями в вакансии. Измени должность на ту, которую ищет соискатель, зарплату и формат работы тоже измени, если они указаны в вакансии. Все остальные блоки должны остаться не изменными, поменяется только нумерация. Ничего не удаляй, не выдумывай и рекомендаций не давай. Обязательно оставь полный стэк умений после блока стажировок."

**СОЗДАЕМ РЕЗЮМЕ В СООТВЕТСТВИИ С ВЫБРАННОЙ ВАКАНСИЕЙ**

In [ ]:
# Пример использования функции корректировки резюме
selected_vacancy_description = selected_vacancy.page_content

# Вызов функции корректировки резюме
adjusted_resume = adjust_resume(generated_resume, selected_vacancy_description)

# Вывод откорректированного резюме
print("Откорректированное резюме:\n")
print(adjusted_resume)

Откорректированное резюме:

Сформированное резюме:

1. ЛИЧНЫЕ ДАННЫЕ:  
Имя: Петров Петр Петрович  
Адрес: Челябинск, ул. Квартальная, д. 1  
Email: ppp1010@gmail.com  
Телефон: +7(789)323-23-45  
Гражданство: РФ  

2. ДОЛЖНОСТЬ:  
Data Scientist  
Зарплата: Обсуждается по результатам собеседования  
Формат работы: Офис  

3. КЛЮЧЕВЫЕ НАВЫКИ:  
- Знание Python, включая библиотеки для машинного обучения и глубокого обучения: TensorFlow, Keras, PyTorch, Scikit-learn  
- Опыт разработки и внедрения алгоритмов машинного обучения, включая компьютерное зрение (CV) и обработку естественного языка (NLP)  
- Опыт работы с основными инструментами для CV и NLP: OpenCV, NLTK, SpaCy, Transformers (Hugging Face)  
- Знание теории вероятностей и математической статистики  
- Опыт оптимизации обучения и работы моделей, контроль их качества  
- Умение разрабатывать прототипы и проектировать архитектуру решений на базе искусственного интеллекта  
- Опыт работы с анализом данных и визуализацией: Pandas, 